In [1]:
example = """32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483"""

example.split('\n')

['32T3K 765', 'T55J5 684', 'KK677 28', 'KTJJT 220', 'QQQJA 483']

# Part 1

In [2]:
from collections import Counter
from enum import Enum
import functools

In [3]:
CARD_ORDER = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']

class Hand(Enum):
    FIVE_OF_A_KIND = 1
    FOUR_OF_A_KIND = 2
    FULL_HOUSE = 3
    THREE_OF_A_KIND = 4
    TWO_PAIR = 5
    ONE_PAIR = 6
    HIGH_CARD = 7

def classify_hand(hand):
    hand_counts_values = sorted(Counter(hand).values())
    # check is 5 of a kind
    if hand_counts_values == [5]:
        return Hand.FIVE_OF_A_KIND
    elif hand_counts_values == [1,4]:
        return Hand.FOUR_OF_A_KIND
    elif hand_counts_values == [2,3]:
        return Hand.FULL_HOUSE
    elif hand_counts_values == [1,1,3]:
        return Hand.THREE_OF_A_KIND
    elif hand_counts_values == [1,2,2]:
        return Hand.TWO_PAIR
    elif hand_counts_values == [1,1,1,2]:
        return Hand.ONE_PAIR
    else:
        return Hand.HIGH_CARD
    
# Returns 1 if hand1 is stronger than hand2, -1 otherwise
def compare_hands(hand1, hand2):
    hand1_class = classify_hand(hand1)
    hand2_class = classify_hand(hand2)
    if hand1_class == hand2_class:
        # go card by card
        for (card1, card2) in zip(hand1, hand2):
            if card1 == card2:
                continue
            return 1 if CARD_ORDER.index(card1) < CARD_ORDER.index(card2) else -1
    return 1 if hand1_class.value < hand2_class.value else -1

hands = [x.split(' ')[0] for x in example.split('\n')]
for hand in hands:
    print('%s: %s' % (hand, classify_hand(hand)))

32T3K: Hand.ONE_PAIR
T55J5: Hand.THREE_OF_A_KIND
KK677: Hand.TWO_PAIR
KTJJT: Hand.TWO_PAIR
QQQJA: Hand.THREE_OF_A_KIND


In [4]:
compare_hands('KTJJT', 'KK677')

-1

In [5]:
def compute_winnings(hands):
    hand_to_bid = {hand: int(bid) for hand, bid in [line.split(' ') for line in hands]}
    sorted_hands = sorted(hand_to_bid.keys(), key=functools.cmp_to_key(compare_hands))
    return sum((i+1) * hand_to_bid[hand] for i, hand in enumerate(sorted_hands))

compute_winnings(example.split('\n'))

6440

In [6]:
real_input = open('day_7.txt').read().split('\n')[:-1]
compute_winnings(real_input)

250058342

# Part 2

In [7]:
def classify_hand_with_jokers(hand):
    if 'J' not in hand:
        return classify_hand(hand)
    
    # figure out what the highest value would be if the jokers were anything
    hand_counts = Counter(hand)
    num_jokers = hand_counts.pop('J')
    
    # degenerate case
    if num_jokers == 5:
        return Hand.FIVE_OF_A_KIND
    
    hand_counts_values = sorted(hand_counts.values())
    counts_plus_jokers = num_jokers + max(hand_counts_values)
    
    if counts_plus_jokers == 5:
        return Hand.FIVE_OF_A_KIND
    
    elif counts_plus_jokers == 4:
        return Hand.FOUR_OF_A_KIND
    
    elif num_jokers == 1 and hand_counts_values == [2,2]:
        return Hand.FULL_HOUSE
    
    elif counts_plus_jokers == 3:
        return Hand.THREE_OF_A_KIND
    
    elif counts_plus_jokers == 2:
        return Hand.ONE_PAIR

    else:
        raise ValueError('should not happen')

hands = [x.split(' ')[0] for x in example.split('\n')]
for hand in hands:
    print('%s: %s' % (hand, classify_hand_with_jokers(hand)))

32T3K: Hand.ONE_PAIR
T55J5: Hand.FOUR_OF_A_KIND
KK677: Hand.TWO_PAIR
KTJJT: Hand.FOUR_OF_A_KIND
QQQJA: Hand.FOUR_OF_A_KIND


In [8]:
JOKER_CARD_ORDER = ['A', 'K', 'Q', 'T', '9', '8', '7', '6', '5', '4', '3', '2', 'J']

# Returns 1 if hand1 is stronger than hand2, -1 otherwise
def compare_hands_with_jokers(hand1, hand2):
    hand1_class = classify_hand_with_jokers(hand1)
    hand2_class = classify_hand_with_jokers(hand2)
    if hand1_class == hand2_class:
        # go card by card
        for (card1, card2) in zip(hand1, hand2):
            if card1 == card2:
                continue
            return 1 if JOKER_CARD_ORDER.index(card1) < JOKER_CARD_ORDER.index(card2) else -1
    return 1 if hand1_class.value < hand2_class.value else -1

In [9]:
def compute_winnings_with_jokers(hands):
    hand_to_bid = {hand: int(bid) for hand, bid in [line.split(' ') for line in hands]}
    sorted_hands = sorted(hand_to_bid.keys(), key=functools.cmp_to_key(compare_hands_with_jokers))
    return sum((i+1) * hand_to_bid[hand] for i, hand in enumerate(sorted_hands))

compute_winnings_with_jokers(example.split('\n'))

5905

In [10]:
compute_winnings_with_jokers(real_input)

250506580